In [ ]:
%pip install filterpy

In [ ]:
import os
import cv2
import numpy as np
from scipy.optimize import linear_sum_assignment
from filterpy.kalman import KalmanFilter

# Load the pre-trained Faster R-CNN model for human detection
try:
    net = cv2.dnn.readNetFromTensorflow(r'E:\Drive D\MA-ICT Convergence\Semester 4\Human-Human-Interaction\src\model\frozen_inference_graph.pb', r'E:\Drive D\MA-ICT Convergence\Semester 4\Human-Human-Interaction\src\model\graph.pbtxt')
except Exception as e:
    print("Error:", e)


# Function to perform human detection
def detect_people(frame, net):
    blob = cv2.dnn.blobFromImage(frame, 0.007843, (frame.shape[1], frame.shape[0]), 127.5)
    net.setInput(blob)
    detections = net.forward()
    boxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
            (startX, startY, endX, endY) = box.astype("int")
            boxes.append((startX, startY, endX, endY))
    return boxes

# Function to initialize Kalman filters for each detected person
def initialize_trackers(boxes):
    trackers = []
    for box in boxes:
        tracker = KalmanFilter(dim_x=4, dim_z=2)
        tracker.F = np.array([[1, 0, 1, 0],
                               [0, 1, 0, 1],
                               [0, 0, 1, 0],
                               [0, 0, 0, 1]])
        tracker.H = np.array([[1, 0, 0, 0],
                               [0, 1, 0, 0]])
        tracker.P *= 10
        tracker.R = np.array([[5, 0],
                               [0, 5]])
        tracker.x = np.array([[box[0], box[1], 0, 0]]).T
        trackers.append(tracker)
    return trackers

# Function to update Kalman filters with detected bounding boxes
def update_trackers(trackers, boxes):
    for tracker, box in zip(trackers, boxes):
        tracker.predict()
        tracker.update(np.array([[box[0] + (box[2] - box[0]) / 2],
                                  [box[1] + (box[3] - box[1]) / 2]]))

# Function to assign detections to tracks using the Hungarian algorithm
def assign_detections_to_tracks(trackers, boxes):
    tracks = np.zeros((len(trackers), 2))
    detections = np.zeros((len(boxes), 2))
    for i, tracker in enumerate(trackers):
        tracks[i] = tracker.x[:2].reshape(2)
    for i, box in enumerate(boxes):
        detections[i] = np.array([(box[0] + box[2]) / 2, (box[1] + box[3]) / 2])
    cost_matrix = np.linalg.norm(tracks[:, None] - detections[None], axis=-1)
    tracker_indices, detection_indices = linear_sum_assignment(cost_matrix)
    return tracker_indices, detection_indices

# Function to perform multiple person tracking
def track_people(video_path, output_dir):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    out = cv2.VideoWriter(output_dir, cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        boxes = detect_people(frame, net)
        
        if len(boxes) > 0:
            if 'trackers' not in locals():
                trackers = initialize_trackers(boxes)
            else:
                update_trackers(trackers, boxes)
                
            for tracker in trackers:
                (x, y), _ = tracker.update(None)
                x, y = int(x), int(y)
                cv2.circle(frame, (x, y), 4, (0, 255, 0), -1)
        
        out.write(frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Define the paths for input and output directories
input_dir = r'E:\Drive D\MA-ICT Convergence\Semester 4\Human-Human-Interaction\dataset\train'
output_dir = r'E:\Drive D\MA-ICT Convergence\Semester 4\Human-Human-Interaction\dataset\output_person_tracking'

# Iterate through all video files in the input directory
for root, dirs, files in os.walk(input_dir):
    for file in files:
        if file.endswith('.avi'):
            video_path = os.path.join(root, file)
            track_people(video_path, output_dir)
